In [6]:
import pathlib
import yaml 
import numpy as np
import sys
import random


# sys.path.remove('/root/repos/polarseg-kitti')
# sys.path.append('~/polarseg-kitti')
# import src.misc.utils as utils

print(sys.path)

if '~/polarseg-kitti' not in sys.path:
    sys.path.append('~/polarseg-kitti')

['/root/repos/polarseg-kitti/notebooks', '/root/miniconda3/lib/python38.zip', '/root/miniconda3/lib/python3.8', '/root/miniconda3/lib/python3.8/lib-dynload', '', '/root/miniconda3/lib/python3.8/site-packages', '~/polarseg-kitti']


In [2]:
def load_SemKITTI_yaml(file, label_name=False):
    # FIXME: rewrite or reference

    with open(file, "r") as stream:
        semkitti_dict = yaml.safe_load(stream)

    # data tuple for unique label values and keys
    if label_name:
        unique_dict = dict()
        # TODO: rewrite or cite this
        for i in sorted(list(semkitti_dict["learning_map"].keys()))[::-1]:
            unique_dict[semkitti_dict["learning_map"][i]] = semkitti_dict["labels"][i]

        unique_keys = np.asarray(sorted(list(unique_dict.keys())))[1:] - 1
        unique_labels = [unique_dict[x] for x in unique_keys + 1]

        return (unique_keys, unique_labels)

    return semkitti_dict

def remap_labels(labels, semkitti_dict):
    for i in labels:
        labels[i] = semkitti_dict["labels"][labels[i]]
    return labels

In [3]:
semkitti_dict = load_SemKITTI_yaml("../semantic-kitti.yaml", True)

semkitti_dict

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18]),
 ['car',
  'bicycle',
  'motorcycle',
  'truck',
  'bus',
  'person',
  'bicyclist',
  'motorcyclist',
  'road',
  'parking',
  'sidewalk',
  'other-ground',
  'building',
  'fence',
  'vegetation',
  'trunk',
  'terrain',
  'pole',
  'traffic-sign'])

In [4]:
with open("../semantic-kitti.yaml", "r") as stream:
    semkitti_dict = yaml.safe_load(stream)

labels = np.fromiter(semkitti_dict["learning_map"].values(), dtype=np.int8)
unique_labels = np.unique(labels)[:-1] # one too much (unlabelled)
unique_names = []
key_list = list(semkitti_dict["learning_map"].keys())

for i in unique_labels:
    unique_names.append(semkitti_dict["labels"][semkitti_dict["learning_map_inv"][i+1]])

unique_names


['car',
 'bicycle',
 'motorcycle',
 'truck',
 'other-vehicle',
 'person',
 'bicyclist',
 'motorcyclist',
 'road',
 'parking',
 'sidewalk',
 'other-ground',
 'building',
 'fence',
 'vegetation',
 'trunk',
 'terrain',
 'pole',
 'traffic-sign']

In [5]:
result = set(semkitti_dict[1]) - set(unique_names)
result

KeyError: 1

In [ ]:
semkitti_dict = load_SemKITTI_yaml("../semantic-kitti.yaml", False)

def find_ignore_label(semkitti_dict: dict):
    for key, value in semkitti_dict["learning_ignore"].items():
        if value == True:
            return key

label = find_ignore_label(semkitti_dict=semkitti_dict)
print(label)

0


In [11]:
choice = random.randint(0,3)

def random_flip(xyz):
    if choice == 0:
        print("No augmentation\n")
    elif choice == 1:
        print("Flip along x-axis\n")
        xyz = -xyz[:,0]
    elif choice == 2:
        print("Flip along y-axis\n")
        xyz = -xyz[:,1]
    elif choice == 3:
        print("Flip along both axis\n")
        xyz_flip1 = -xyz[:,0]
        xyz = -xyz_flip1[:,1]
    return xyz


0

In [12]:
!pip install scipy

     |████████████████████████████████| 41.6 MB 11.8 MB/s eta 0:00:01


In [21]:
import numpy as np
from scipy.ndimage.interpolation import rotate

xyz = np.random.randint(800, 1000, size=[3, 3])

print(xyz)

rotated = rotate()

print(rotated)

rotate_rad = np.deg2rad(45)
c, s = np.cos(rotate_rad), np.sin(rotate_rad)
j = np.matrix([[c, s], [-s, c]])
xyz[:,:2] = np.dot( xyz[:,:2],j)

print(xyz)

[[916 886 818]
 [987 818 838]
 [953 903 807]]
[[  0   0   0   0]
 [  0 871 827   0]
 [  0 959 886   0]
 [  0   0   0   0]]
[[  21 1274  818]
 [ 119 1276  838]
 [  35 1312  807]]


/tmp/ipykernel_492952/2878214580.py:2: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import rotate
